In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd

data = pd.read_csv('df_clean (1).csv') ##/content/df_clean.csv
data.head()

,Unnamed: 0,Table_title,concept_1,predicate,concept_2
0,0,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,has clinical finding,mild elevation ALT/AST level
1,1,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,has clinical finding,moderate elevation GGT level
2,2,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,has clinical finding,marked elevation ALP level
3,3,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,has clinical finding,marked elevation ALT/AST level
4,4,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,has clinical finding,mild elevation GGT level


In [ ]:
data.shape

(1686, 5)

In [ ]:
import string

def clean_string(text):
    # Split text into words
    words = text.split()
    clean_words = []
    for word in words:
        # Remove punctuation and digits from word
        word = word.translate(str.maketrans('', '', string.punctuation + string.digits))
        # Only keep words with more than one character
        if len(word) > 1:
            clean_words.append(word)
    # Join cleaned words into a single string
    clean_text = ' '.join(clean_words)
    return clean_text

In [ ]:
data = data.fillna('')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Create a label encoder
encoder = LabelEncoder()

# Fit the encoder on the predicate column
encoder.fit(data['predicate'])

# Transform the predicate column
predicate = encoder.transform(data['predicate'])

In [ ]:
data['predicate'].value_counts()

has cause                                   681
has clinical finding                        427
has indication                              170
has positive lab value                      159
has topological description/distribution     67
has risk factor                              42
Has clinical finding                         31
Has drug                                     19
Has indication                               19
Has time                                     16
Has function                                 15
has abnormality                              11
Has morphology                                8
is risk factor                                8
Has time|clinical finding                     5
Has quality                                   4
has diagnosis                                 3
has cause                                     1
Name: predicate, dtype: int64

In [ ]:
import torch
print(len((predicate)))
# predicate = encoder.transform(data['predicate'])

labels = torch.tensor(predicate)
print(len(set(predicate)))

1686
18


In [ ]:
#Oversampling 

In [ ]:
df_x = data.drop(columns = ['predicate'])
df_y = data['predicate']

In [ ]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(random_state = 42)
df_x_ov, df_y_ov = oversample.fit_resample(df_x, df_y)

In [ ]:
# the below code should be run only when undersampling is required 
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(random_state = 42)
df_x_ov, df_y_ov = undersample.fit_resample(df_x, df_y)

In [ ]:
df_y_ov.value_counts()

has clinical finding                        681
Has function                                681
has cause                                   681
has diagnosis                               681
is risk factor                              681
has indication                              681
has abnormality                             681
Has quality                                 681
Has time                                    681
has positive lab value                      681
Has clinical finding                        681
Has indication                              681
Has drug                                    681
Has time|clinical finding                   681
Has morphology                              681
has topological description/distribution    681
has cause                                   681
has risk factor                             681
Name: predicate, dtype: int64

In [ ]:
df_x_ov

,Unnamed: 0,Table_title,concept_1,concept_2
0,0,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,mild elevation ALT/AST level
1,1,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,moderate elevation GGT level
2,2,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,marked elevation ALP level
3,3,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,marked elevation ALT/AST level
4,4,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,mild elevation GGT level
...,...,...,...,...
12253,649,Trigger factors for seizures,Seizures,Intercurrent infections and metabolic disturba...
12254,652,Trigger factors for seizures,Seizures,Missed doses of antiepileptic drugs in treated...
12255,646,Trigger factors for seizures,Seizures,Recreational drug misuse
12256,650,Trigger factors for seizures,Seizures,"Uncommon: loud noises, music, reading, hot baths"


In [ ]:
combined_df = pd.concat([df_x_ov, df_y_ov], axis = 1)

combined_df

,Unnamed: 0,Table_title,concept_1,concept_2,predicate
0,0,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,mild elevation ALT/AST level,has clinical finding
1,1,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,moderate elevation GGT level,has clinical finding
2,2,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,marked elevation ALP level,has clinical finding
3,3,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,marked elevation ALT/AST level,has clinical finding
4,4,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,mild elevation GGT level,has clinical finding
...,...,...,...,...,...
12253,649,Trigger factors for seizures,Seizures,Intercurrent infections and metabolic disturba...,is risk factor
12254,652,Trigger factors for seizures,Seizures,Missed doses of antiepileptic drugs in treated...,is risk factor
12255,646,Trigger factors for seizures,Seizures,Recreational drug misuse,is risk factor
12256,650,Trigger factors for seizures,Seizures,"Uncommon: loud noises, music, reading, hot baths",is risk factor


In [ ]:
#combined_df.to_csv('/content/combined.csv')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Set the number of classes
num_classes = 18 # Set this to the number of classes in your data
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=num_classes)
# model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Create a label encoder
encoder = LabelEncoder()

# Fit the encoder on the predicate column
encoder.fit(combined_df['predicate'])

# Transform the predicate column
predicate = encoder.transform(combined_df['predicate'])

In [ ]:
# Prepare the data

# Convert the columns to strings
combined_df['Table_title'] = combined_df['Table_title'].astype(str).apply(clean_string)
combined_df['concept_1'] = combined_df['concept_1'].astype(str).apply(clean_string)
combined_df['concept_2'] = combined_df['concept_2'].astype(str).apply(clean_string)

# Tokenize the text
text = combined_df['Table_title'] + ' ' + combined_df['concept_1'] + ' ' + combined_df['concept_2']
inputs = tokenizer(text.tolist(), return_tensors="pt", padding=True, truncation=True)
# Create the labels tensor
labels = torch.tensor(predicate)
# Convert the inputs to a PyTorch tensor
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Create a DataLoader
batch_size = 8 # Reduce the batch size
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size)

# Set the learning rate and gradient accumulation steps
learning_rate = 1e-5
gradient_accumulation_steps = 4

# Create the optimizer
optimizer = Adam(model.parameters(), lr=learning_rate)

# Fine-tune the model
model.train()
for epoch in range(3):
    for i, (input_ids, attention_mask, labels) in enumerate(dataloader):
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps
        loss.backward()

        if (i + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyboardInterrupt: ignored